# Calculate and save the matrices in RAS space

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import svd
import numpy as np
import math
from scipy import interpolate
import random
from numpy.linalg import inv
import scipy.io
import pydicom
import cmath
from scipy.spatial.transform import Rotation as R
import os
import datetime
import csv
import datetime

In [ ]:
from functions import *

In [ ]:
# define GA and no_GA subjects:
GA = ['2017-03-31_ZA', '2017-04-07_SK', '2017-06-02_MN', '2017-06-16_SJAS', '2017-06-23_AKDW', '2017-07-07_WHO', '2017-11-24_JA', '2018-05-25_MT', '2019-01-11_MS_0607175918', '2015-04-14_EB', '2015-05-19_EV', '2016-09-30_AMRH', '2016-10-21_JNT', '2016-12-09_MN', '2017-02-03_PL', '2017-03-10_SJAS', '2017-03-17_MTD', '2017-03-28_NPN']
no_GA = ['2015-09-01_TSS', '2015-10-06_RMI', '2015-10-09_SK', '2015-12-01_FCA', '2015-12-01_VTB', '2015-12-15_LR', '2015-12-15_PN', '2015-12-22_SRo', '2016-02-26_MR', '2016-03-22_NLL', '2016-04-05_ESS', '2016-04-05_JMPD', '2016-04-26_JJ', '2016-05-20_NLL', '2016-05-31_BNP', '2016-06-21_AHA', '2016-06-28_MS', '2016-06-28_NLVS', '2016-08-05_EJ', '2016-09-30_VJ', '2016-10-04_JMPD', '2016-10-11_JJ', '2016-12-13_OKR', '2017-01-06_NM', '2017-01-17_JVJ', '2017-02-14_SLA', '2017-03-14_ESS', '2017-06-06_MS', '2017-06-06_RL', '2017-06-13_LR', '2017-06-20_OH', '2017-09-05_NSH', '2017-09-05_SE2', '2017-09-12_ICV', '2017-09-26_MS', '2017-10-03_RA', '2017-10-10_MA', '2018-01-02_ACK', '2018-01-09_NSN', '2018-03-16_EA', '2018-03-16_TP', '2018-04-17_MWA', '2018-04-24_LR']
subjs = np.append(no_GA, GA)
print(subjs)

# define output folder:
folder = '/pc_disk1/moco/StudentProjects/MSc/Hannah/FrontRadio_Eichhorn_2021/analysis/'

In [ ]:
print(len(subjs))

In [ ]:
# define scan times for each sequence:
sequ = ['Head_t1_mprage_sag_p2_iso', 'Head_t1_tirm_tra_darkfluid_p2', 'Head_t2_tirm_tra_darkfluid_p2', 'Head_t2_tirm_cor_darkfluid_4mm', 'Head_t2_blade_tra_5mm_GD', 'Head_t1_mprage_sag_p2_iso_GD']
sequ_abbr = ['t1_mprage_sag', 't1_tirm_tra', 't2_tirm_tra', 't2_tirm_cor', 't2_blade_tra_5mm_GD', 't1_mprage_sag*GD']
scantimes = [266, 174, 272, 164, 122, 266]   # in seconds from TCL_mMR_documentation_BUF.xlsx

In [ ]:
# get scan start for each sequ type and subject:
save = ['Subj_ID, sequence, start_time, end_time']
for sub in subjs:
    for seq, sct in zip(sequ_abbr, scantimes):
        path = '/mnt/mocodata1/BUF/DataMMR/'+sub+'/PETMR*/*'+seq+'*/*'
        find = glob.glob(path)
        if seq == 't1_mprage_sag':
            if len(find)>1:
                find = [s for s in find if "GD" not in s]
        if len(find) > 0:
            file = find[0]
            start_time = getAcqTimeFromDICOM(file)
            # divide sct in minutes and seconds:
            mins = sct//60
            scs = sct%60
            start_tim = datetime.datetime.strptime(start_time, '%H%M%S.%f')
            timeToAdd = datetime.timedelta(minutes=mins, seconds=scs)
            end_time = (start_tim+ timeToAdd).strftime('%H%M%S.%f')
            save.append(sub+', '+seq+', '+str(start_time)+', '+str(end_time))
save = np.array(save)
np.savetxt(folder+'Scan_Times.txt', save, fmt="%s")

In [ ]:
# load scan start for each sequence:
Scan_Times = np.loadtxt(folder+'Scan_Times.txt', dtype=str, delimiter=', ', skiprows=1)

Calculate matrices with reference position in middle of MPRAGE:

In [ ]:
# file with dicom info (id,age,scan start)
file = folder+"/dicomData.csv"
path = "/mnt/mocodata1/BUF/DataTCL/"
A_dcs2ecs=np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

for d in no_GA:
    aln, tim, poa, log  = getFileNames(path+d)
    print(aln,tim,poa,log)
    A_tcs2dcs = readtcs2dcs(aln)
    A_tcs2ecs = A_dcs2ecs@A_tcs2dcs
    time = getTime(file,d)
    print(time,d)
    # add half of MPRAGE acquisition time (266s)
    timeToAdd = datetime.timedelta(minutes=2, seconds=13)
    time_ = datetime.datetime.strptime(time, '%H%M%S.%f')
    time = (time_+ timeToAdd).strftime('%H%M%S.%f')
    refFrame = 0
    # get the ref frame (start of scan time) either from tim or log file
    if tim!="":
        print("IN TIM")
        timeDifference = getTimeDifference(log)
        print(timeDifference)
        timeDiffList = timeDifference.split(":")
        a = timeDiffList[-1].split(".")
        # convert to python timedate
        timeToAdd = datetime.timedelta(minutes=int(timeDiffList[1]), seconds=int(a[0]), milliseconds=int(a[1]))
        refFrame = findFrameNumber(tim, time, timeToAdd)
        print(refFrame)
    elif log!="":
        print("IN LOG")
        print(time)
        refFrame = findFrameNumberInLog(log, time)
    else:
        break
    if not os.path.exists(folder+"No GA/matricesREF "+d):
        os.mkdir(folder+"No GA/matricesREF "+d)
    refTr = transform(poa, A_tcs2ecs, refFrame, folder+"No GA/matricesREF "+d)
    print(time,refFrame,refTr)

In [ ]:
# file with dicom info (id,age,scan start)
file = folder+"dicomData.csv"
path = "/mnt/mocodata1/BUF/DataTCL/"
A_dcs2ecs=np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

for d in GA:
    aln, tim, poa, log  = getFileNames(path+d)
    print(aln,tim,poa,log)
    A_tcs2dcs = readtcs2dcs(aln)
    A_tcs2ecs = A_dcs2ecs@A_tcs2dcs
    #time = getAcqTimeFromDICOM(dcm)
    time = getTime(file,d)
    print(time,d)
    # add half of MPRAGE acquisition time (266s)
    timeToAdd = datetime.timedelta(minutes=2, seconds=13)
    time_ = datetime.datetime.strptime(time, '%H%M%S.%f')
    time = (time_+ timeToAdd).strftime('%H%M%S.%f')
    refFrame = 0
    # get the ref frame (start of scan time) either from tim or log file
    if tim!="":
        print("IN TIM")
        timeDifference = getTimeDifference(log)
        print(timeDifference)
        timeDiffList = timeDifference.split(":")
        a = timeDiffList[-1].split(".")
        # convert to python timedate
        timeToAdd = datetime.timedelta(minutes=int(timeDiffList[1]), seconds=int(a[0]), milliseconds=int(a[1]))
        refFrame = findFrameNumber(tim, time, timeToAdd)
        print(refFrame)
    elif log!="":
        print("IN LOG")
        print(time)
        refFrame = findFrameNumberInLog(log, time)
    else:
        break
    if not os.path.exists(folder+"GA/matricesREF "+d):
        os.mkdir(folder+"GA/matricesREF "+d)
    refTr = transform(poa, A_tcs2ecs, refFrame, folder+"GA/matricesREF "+d)
    print(time,refFrame,refTr)

# Analysis for excluded children:

In [ ]:
subjs = ['2015-04-14_NJ', '2015-04-24_NJ', '2015-05-05_SB', '2015-05-08_ADS', '2015-05-26_PN', '2015-11-03_ELV', '2015-11-13_PL', '2016-01-05_JJW', '2016-02-26_FLTH', '2016-03-11_MBH', '2016-04-01_NLL', '2016-05-03_HF', '2016-05-03_MBH', '2016-05-24_RGR2', '2016-07-08_JK', '2016-08-02_JJ', '2016-10-19_BA', '2016-12-16_OKR', '2017-01-24_JD', '2017-02-24_HGSF', '2017-03-03_BA', '2017-04-21_ATWJ', '2017-05-05_AJ', '2017-05-19_FSH', '2017-08-22_SB', '2017-08-29_SEKB', '2017-09-01_MK', '2017-10-10_SB', '2017-11-03_VWN', '2017-11-28_ToM', '2015-11-10_BSA']
print(len(subjs))

In [ ]:
# go through GA list and sort subjs:
GA_list = ['2015-04-14_NJ', '2015-04-24_NJ',  '2015-05-05_SB',  '2016-07-08_JK',  '2016-10-19_BA',  '2017-02-24_HGSF', '2017-03-03_BA', '2017-04-21_ATWJ', '2017-05-05_AJ', '2017-09-01_MK', '2015-05-08_ADS']
noGA_list = ['2015-11-10_BSA', '2017-11-28_ToM', '2015-05-26_PN', '2015-11-03_ELV', '2015-11-13_PL', '2016-02-26_FLTH', '2016-03-11_MBH',  '2016-04-01_NLL', '2016-05-03_MBH',  '2017-01-24_JD', '2017-05-19_FSH', '2017-08-22_SB', '2017-08-29_SEKB', '2017-10-10_SB', '2016-05-03_HF', '2016-08-02_JJ', '2017-11-03_VWN']


In [ ]:
# get scan start for each sequ type and subject:
save = ['Subj_ID, sequence, start_time, end_time']
for sub in subjs:
    for seq, sct in zip(sequ_abbr, scantimes):
        path = '/mnt/mocodata1/BUF/DataMMR/'+sub+'/PETMR*/*'+seq+'*/*'
        find = glob.glob(path)
        if seq == 't1_mprage_sag':
            if len(find)>1:
                find = [s for s in find if "GD" not in s]
        if len(find) > 0:
            file = find[0]
            start_time = getAcqTimeFromDICOM(file)
            # divide sct in minutes and seconds:
            mins = sct//60
            scs = sct%60
            start_tim = datetime.datetime.strptime(start_time, '%H%M%S.%f')
            timeToAdd = datetime.timedelta(minutes=mins, seconds=scs)
            end_time = (start_tim+ timeToAdd).strftime('%H%M%S.%f')
            save.append(sub+', '+seq+', '+str(start_time)+', '+str(end_time))
save = np.array(save)
np.savetxt(folder+'Scan_Times_Excl.txt', save, fmt="%s")

In [ ]:
# load start time for each scan:
Scan_Times = np.loadtxt(folder+'Scan_Times_Excl.txt', dtype=str, delimiter=', ', skiprows=1)

In [ ]:
# file with dicom info (id,age,scan start)
file = folder+"dicomData.csv"
path = "/mnt/mocodata1/BUF/DataTCL/"
A_dcs2ecs=np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

for d in noGA_list:
    aln, tim, poa, log  = getFileNames(path+d)
    print(aln,tim,poa,log)
    A_tcs2dcs = readtcs2dcs(aln)
    A_tcs2ecs = A_dcs2ecs@A_tcs2dcs
    time = getTime(file,d)
    print(time,d)
    # add half of MPRAGE acquisition time (266s)
    timeToAdd = datetime.timedelta(minutes=2, seconds=13)
    time_ = datetime.datetime.strptime(time, '%H%M%S.%f')
    time = (time_+ timeToAdd).strftime('%H%M%S.%f')
    refFrame = 0
    # get the ref frame (start of scan time) either from tim or log file
    if tim!="":
        print("IN TIM")
        timeDifference = getTimeDifference(log)
        print(timeDifference)
        timeDiffList = timeDifference.split(":")
        a = timeDiffList[-1].split(".")
        # convert to python timedate
        timeToAdd = datetime.timedelta(minutes=int(timeDiffList[1]), seconds=int(a[0]), milliseconds=int(a[1]))
        refFrame = findFrameNumber(tim, time, timeToAdd)
    elif log!="":
        print("IN LOG")
        print(time)
        refFrame = findFrameNumberInLog(log, time)
    else:
        break
    if not os.path.exists(folder+"No GA_Excl/matricesREF "+d):
        os.mkdir(folder+"No GA_Excl/matricesREF "+d)
    refTr = transform(poa, A_tcs2ecs, refFrame, "./new_calc/No GA_Excl/matricesREF "+d)
    if np.sum(refTr)==0:
        print(refFrame, ' add 1')
        refFrame = str(int(refFrame)+1)
        refTr = transform(poa, A_tcs2ecs, refFrame, folder+"No GA_Excl/matricesREF "+d)
    print(time,refFrame,refTr)

In [ ]:
# file with dicom info (id,age,scan start)
file = folder+"dicomData.csv"
path = "/mnt/mocodata1/BUF/DataTCL/"
A_dcs2ecs=np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

for d in GA_list:
    aln, tim, poa, log  = getFileNames(path+d)
    print(aln,tim,poa,log)
    A_tcs2dcs = readtcs2dcs(aln)
    A_tcs2ecs = A_dcs2ecs@A_tcs2dcs
    time = getTime(file,d)
    print(time,d)
    # add half of MPRAGE acquisition time (266s)
    timeToAdd = datetime.timedelta(minutes=2, seconds=13)
    time_ = datetime.datetime.strptime(time, '%H%M%S.%f')
    time = (time_+ timeToAdd).strftime('%H%M%S.%f')
    refFrame = 0
    # get the ref frame (start of scan time) either from tim or log file
    if tim!="":
        print("IN TIM")
        timeDifference = getTimeDifference(log)
        print(timeDifference)
        timeDiffList = timeDifference.split(":")
        a = timeDiffList[-1].split(".")
        # convert to python timedate
        timeToAdd = datetime.timedelta(minutes=int(timeDiffList[1]), seconds=int(a[0]), milliseconds=int(a[1]))
        refFrame = findFrameNumber(tim, time, timeToAdd)
        print(refFrame)
    elif log!="":
        print("IN LOG")
        print(time)
        refFrame = findFrameNumberInLog(log, time)
    else:
        break
    if not os.path.exists(folder+"GA_Excl/matricesREF "+d):
        os.mkdir(folder+"GA_Excl/matricesREF "+d)
    refTr = transform(poa, A_tcs2ecs, refFrame, folder+"GA_Excl/matricesREF "+d)
    print(time,refFrame,refTr)